# Michelson tutorial
## Chapter two
This chapter introduces types in Michelson, advanced stack usage and stack protection instructions

##### 1. Types in Michelson

Michelson is a strongly typed language with no type inference, which means that you must always indicates which type you are using to the compiler. The compiler will then checks that all the values used in the contract are correctly typed.  
For example, if you write `PUSH int 3 ; PUSH string "hello" ; ADD ;`, the compiler will throw an error as you cannot add an integer and a string together:

In [8]:
## THIS DOESN'T WORK!
storage unit ;
parameter unit ;
code {
    DROP ;
    PUSH int 3 ;
    PUSH string "Hello";
    ADD ;
    NIL operation ;
    PAIR
} ;
RUN %default Unit Unit ;

storage unit;
parameter unit;
code { DROP ; PUSH int 3 ; PUSH string "Hello" ; ADD ; NIL operation ; PAIR };
RUN: use %default; drop all; push (Unit, Unit);
DROP: pop (Unit, Unit);
PUSH: push 3;
PUSH: push Hello;
ADD: pop Hello, 3;

MichelsonRuntimeError: unsupported argument types String and Int
at RUN -> ADD

It is also worth pointing out that the values you use on the stack are immutable, which means that you cannot "modify" them, you must remove them, add them or concatenate them together to change them.

You can find below a list of the most common types in Michelson:
- Core data types:  
  - __int__ (positive and negative numbers) 
  - __nat__ (positive numbers) 
  - __string__ (one or multiple characters) 
  - __bytes__
  - __pair__ (a pair of two values)
  - __bool__ (boolean value)
  - __unit__ (a placeholder type when no value or parameter is required)
  - __option__ (optional value with 2 possible values: _SOME (type)_ and _NONE_)
  - __list__ (a list of ordered values of the same type)
  - __set__ (a list of unordered unique values of the same type)
  - __map__ (a list of key/value pairs)
  - __big_map__ (a lazily deserialized list of key/value pairs, used for large amounts of data)
- Domain specific data types:
  - __timestamp__ (dates in ISO 8601 format)
  - __mutez__ (Tezos tokens type)
  - __address__ (Tezos addresses)
  - __operation__ (internal operation emitted by a contract)
  - __key__ (public cryptographic key)

Some of these types are called [__comparable types__](https://tezos.gitlab.io/whitedoc/michelson.html#full-grammar) which means that the comparison of two values of these types will yield an _integer_ that you can then use to know if they are equal or if one is lower or higher than the other.  
For example, *int*, *nat*, *string*, *bytes*, *mutez*, *bool*, *timestamp* and *address* are all comparable types.

##### 2. Advanced stack usage

###### - Inline code

In [24]:
storage string ;
parameter unit ;
code {
    DUP ;
    CAR ;
    SWAP ;
    CDR ;
    PUSH string "world" ;
    SWAP ;
    CONCAT;
    DIP { DROP } ;
    NIL operation ;
    PAIR
}

storage string;
parameter unit;
code { DUP ; CAR ; SWAP ; CDR ; PUSH string "world" ; SWAP ; CONCAT ; DIP { DROP } ; NIL operation ; PAIR };

In [25]:
RUN %default Unit "Hello " ;

RUN: use %default; drop all; push (Unit, 'Hello ');
  DUP: push (Unit, 'Hello ');
  CAR: pop (Unit, 'Hello '); push Unit;
  SWAP: pop Unit, (Unit, 'Hello '); push Unit; push (Unit, 'Hello ');
  CDR: pop (Unit, 'Hello '); push Hello ;
  PUSH: push world;
  SWAP: pop world, Hello ; push world; push Hello ;
  CONCAT: pop Hello ; pop world; push Hello world;
  DIP: protect 1 item(s);
    DROP: pop Unit;
    restore 1 item(s);
  NIL: push [];
  PAIR: pop [], Hello world; push ([], 'Hello world');

value,type
"""Hello world""",string


###### - Step-by-step execution

In [7]:
storage string ;
parameter unit ;

storage string;
parameter unit;

In [8]:
BEGIN Unit "Hello " ;

value,type
"Pair Unit ""Hello """,pair unit string


In [9]:
DUP ; DUMP ;

value,type
"Pair Unit ""Hello """,pair unit string
"Pair Unit ""Hello """,pair unit string


In [10]:
CAR ; DUMP ;

value,type
Unit,unit
"Pair Unit ""Hello """,pair unit string


In [11]:
SWAP ; DUMP ;

value,type
"Pair Unit ""Hello """,pair unit string
Unit,unit


In [12]:
CDR ; DUMP ;

value,type
"""Hello """,string
Unit,unit


In [13]:
PUSH string "world" ; DUMP ;

value,type
"""world""",string
"""Hello """,string
Unit,unit


In [14]:
SWAP ; DUMP ;

value,type
"""Hello """,string
"""world""",string
Unit,unit


In [15]:
CONCAT ; DUMP ;

value,type
"""Hello world""",string
Unit,unit


In [16]:
DIP { DROP } ; DUMP ;

DIP: protect 1 item(s);
  DROP: pop Unit;
  restore 1 item(s);

value,type
"""Hello world""",string


In [17]:
NIL operation ;

value,type
[],list operation


In [18]:
PAIR ;

value,type
"Pair {} ""Hello world""",pair (list operation) string


In [19]:
COMMIT ;

value,type
"""Hello world""",string
